In [1]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/alyssa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [7]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [8]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [9]:
processed_docs = documents['headline_text'].map(preprocess)

In [10]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [11]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [12]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [13]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

In [14]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3567 ("compulsori") appears 1 time.
Word 3568 ("ratepay") appears 1 time.


In [15]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [16]:
corpus_tfidf = tfidf[bow_corpus]

In [17]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [19]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"south" + 0.025*"hous" + 0.022*"north" + 0.019*"bushfir" + 0.018*"miss" + 0.017*"hospit" + 0.016*"investig" + 0.014*"west" + 0.012*"dead" + 0.012*"coast"
Topic: 1 
Words: 0.032*"kill" + 0.023*"shoot" + 0.019*"attack" + 0.017*"bank" + 0.015*"protest" + 0.015*"polic" + 0.014*"offic" + 0.013*"assault" + 0.013*"chines" + 0.011*"michael"
Topic: 2 
Words: 0.058*"australia" + 0.047*"australian" + 0.027*"world" + 0.018*"canberra" + 0.017*"test" + 0.015*"island" + 0.011*"open" + 0.011*"farm" + 0.011*"win" + 0.011*"return"
Topic: 3 
Words: 0.031*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"murder" + 0.019*"death" + 0.017*"face" + 0.016*"alleg" + 0.016*"woman" + 0.015*"jail" + 0.015*"accus"
Topic: 4 
Words: 0.017*"say" + 0.017*"chang" + 0.014*"speak" + 0.014*"power" + 0.013*"interview" + 0.012*"worker" + 0.011*"climat" + 0.011*"concern" + 0.011*"polit" + 0.010*"flood"
Topic: 5 
Words: 0.022*"market" + 0.020*"news" + 0.019*"women" + 0.018*"live" + 0.013*"rise" + 0.013*"h

In [20]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [21]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.017*"charg" + 0.017*"polic" + 0.014*"murder" + 0.011*"court" + 0.010*"woman" + 0.010*"alleg" + 0.010*"jail" + 0.009*"death" + 0.009*"arrest" + 0.008*"shoot"
Topic: 1 Word: 0.012*"govern" + 0.010*"rural" + 0.008*"elect" + 0.008*"nation" + 0.007*"health" + 0.007*"fund" + 0.006*"budget" + 0.006*"say" + 0.006*"feder" + 0.006*"labor"
Topic: 2 Word: 0.013*"coast" + 0.013*"south" + 0.013*"north" + 0.010*"australia" + 0.009*"gold" + 0.008*"west" + 0.007*"christma" + 0.007*"korea" + 0.007*"sport" + 0.006*"festiv"
Topic: 3 Word: 0.009*"john" + 0.008*"leagu" + 0.007*"cricket" + 0.007*"rugbi" + 0.006*"australia" + 0.006*"syria" + 0.005*"shorten" + 0.005*"thousand" + 0.005*"world" + 0.004*"final"
Topic: 4 Word: 0.013*"stori" + 0.008*"search" + 0.006*"social" + 0.006*"tree" + 0.006*"ash" + 0.006*"alan" + 0.006*"territori" + 0.006*"america" + 0.006*"northern" + 0.006*"wild"
Topic: 5 Word: 0.018*"donald" + 0.016*"drum" + 0.010*"tuesday" + 0.010*"monday" + 0.006*"hong" + 0.006*"kong" +

In [22]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7252238392829895	 
Topic: 0.012*"govern" + 0.010*"rural" + 0.008*"elect" + 0.008*"nation" + 0.007*"health" + 0.007*"fund" + 0.006*"budget" + 0.006*"say" + 0.006*"feder" + 0.006*"labor"

Score: 0.17474038898944855	 
Topic: 0.030*"trump" + 0.010*"market" + 0.009*"share" + 0.008*"david" + 0.008*"morrison" + 0.008*"wall" + 0.008*"scott" + 0.007*"juli" + 0.007*"australian" + 0.006*"octob"

Score: 0.012504851445555687	 
Topic: 0.020*"countri" + 0.015*"hour" + 0.012*"live" + 0.009*"thursday" + 0.007*"violenc" + 0.006*"action" + 0.006*"farm" + 0.006*"domest" + 0.006*"june" + 0.006*"result"

Score: 0.012504762038588524	 
Topic: 0.013*"stori" + 0.008*"search" + 0.006*"social" + 0.006*"tree" + 0.006*"ash" + 0.006*"alan" + 0.006*"territori" + 0.006*"america" + 0.006*"northern" + 0.006*"wild"

Score: 0.012504600919783115	 
Topic: 0.013*"royal" + 0.010*"friday" + 0.010*"commiss" + 0.008*"sexual" + 0.007*"tasmanian" + 0.006*"jam" + 0.006*"kid" + 0.006*"track" + 0.006*"cancer" + 0.006*"foota

In [23]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.41942426562309265	 Topic: 0.022*"market" + 0.020*"news" + 0.019*"women" + 0.018*"live" + 0.013*"rise"
Score: 0.28086942434310913	 Topic: 0.017*"say" + 0.017*"chang" + 0.014*"speak" + 0.014*"power" + 0.013*"interview"
Score: 0.1829468011856079	 Topic: 0.031*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"murder" + 0.019*"death"
Score: 0.01668371632695198	 Topic: 0.034*"elect" + 0.018*"water" + 0.017*"state" + 0.015*"tasmanian" + 0.011*"liber"
Score: 0.01668291725218296	 Topic: 0.019*"donald" + 0.014*"nation" + 0.014*"farmer" + 0.014*"plan" + 0.013*"rural"
Score: 0.016680052503943443	 Topic: 0.025*"south" + 0.025*"hous" + 0.022*"north" + 0.019*"bushfir" + 0.018*"miss"
Score: 0.016679983586072922	 Topic: 0.030*"govern" + 0.018*"feder" + 0.016*"warn" + 0.014*"countri" + 0.014*"fund"
Score: 0.016677621752023697	 Topic: 0.032*"kill" + 0.023*"shoot" + 0.019*"attack" + 0.017*"bank" + 0.015*"protest"
Score: 0.016677621752023697	 Topic: 0.058*"australia" + 0.047*"australian" + 0.027*"w